In [1]:
import os
from music21 import converter, pitch, interval
import numpy as np

In [2]:
# Define save directory
save_dir = 'mozart/'

In [3]:
# Identify list of MIDI files
songList = os.listdir(save_dir)
songList

['.DS_Store',
 'piano_sonata_282_(hisamori).mid',
 'piano_sonata_280_(hisamori).mid',
 'piano_sonata_279_(hisamori).mid',
 'piano_sonata_281_(hisamori).mid']

In [4]:
songList.remove('.DS_Store')
#songList.remove('brahms_opus1_4.mid')
#songList

In [5]:
# Create empty list for scores
originalScores = []

In [6]:
# Load and make list of stream objects
for song in songList:
    score = converter.parse(save_dir+song)
    print(song + 'worked')
    originalScores.append(score)

piano_sonata_282_(hisamori).midworked
piano_sonata_280_(hisamori).midworked
piano_sonata_279_(hisamori).midworked
piano_sonata_281_(hisamori).midworked


In [7]:
from music21 import instrument

# Define function to test whether stream is monotonic
def monophonic(stream):
    try:
        length = len(instrument.partitionByInstrument(stream).parts)
    except:
        length = 0
    return length == 1

In [8]:
# Merge notes into chords
originalScores = [song.chordify() for song in originalScores]

In [9]:
from music21 import note, chord

# Define empty lists of lists
originalChords = [[] for _ in originalScores]
originalDurations = [[] for _ in originalScores]
originalKeys = []

In [10]:
# Extract notes, chords, durations, and keys
for i, song in enumerate(originalScores):
    originalKeys.append(str(song.analyze('key')))
    for element in song:
        if isinstance(element, note.Note):
            originalChords[i].append(element.pitch)
            originalDurations[i].append(element.duration.quarterLength)
        elif isinstance(element, chord.Chord):
            originalChords[i].append('.'.join(str(n) for n in element.pitches))
            originalDurations[i].append(element.duration.quarterLength)
    print(str(i))

0
1
2
3


In [11]:
originalKeys

['E- major', 'F major', 'C major', 'B- major']

In [109]:
# Create list of chords and durations from songs in C major
cMajorChords = [c for (c, k) in zip(originalChords, originalKeys) if (k == 'C major')]
cMajorDurations = [c for (c, k) in zip(originalDurations, originalKeys) if (k == 'C major')]
len(cMajorChords)

1

In [110]:
# Map unique chords to integers
uniqueChords = np.unique([i for s in originalChords for i in s])
chordToInt = dict(zip(uniqueChords, list(range(0, len(uniqueChords)))))

In [111]:
# Print number of unique notes and chords
print(len(uniqueChords))

2913


In [112]:
# Map unique durations to integers
uniqueDurations = np.unique([i for s in originalDurations for i in s])
durationToInt = dict(zip(uniqueDurations, list(range(0, len(uniqueDurations)))))

In [113]:
# Print number of unique durations
print(len(uniqueDurations))

16


In [114]:
uniqueChords

array(['A1', 'A1.A2', 'A1.A2.B-4', ..., 'G5.B-5.C6', 'G5.B5', 'G5.C6'],
      dtype='<U27')

In [115]:
cMajorChords

[['C3.E4.G4',
  'C3.C4.C5',
  'C3.B3',
  'C3.C4',
  'C3',
  'C3.C5',
  'B4',
  'D5',
  'C5',
  'E5',
  'D5',
  'F5',
  'E5',
  'G5',
  'F5',
  'F2.F3.A4.D5',
  'F2.F3.F5.A5',
  'F2.F3.A5',
  'G5',
  'F5',
  'E5',
  'D5',
  'C5.D5',
  'B4.C5',
  'E3.C5',
  'E5',
  'F3.D5',
  'F5',
  'G3.B4',
  'D5',
  'C3.E4.G4',
  'C3.C4.C5',
  'C3.B3.C5',
  'C3.C4.C5',
  'C3',
  'C3.C5',
  'B4',
  'D5',
  'C5',
  'E5',
  'D5',
  'F5',
  'E5',
  'G5',
  'F5',
  'F2.F3.A4.D5',
  'F2.F3.F5.A5',
  'F2.F3.A5',
  'G5',
  'F5',
  'E5',
  'D5',
  'C5.D5',
  'B4.C5',
  'E3.C5',
  'E5',
  'F3.D5',
  'F5',
  'G3.B4',
  'D5',
  'C4.C5',
  'C4.G4',
  'C4.G4.D5',
  'C4.E4.C5.D5',
  'C4.E4.C5',
  'G4.B4',
  'C4.C5',
  'C4.G4',
  'C4.E4.E5',
  'G4',
  'B3.C#5',
  'B3.G4.C#5',
  'B3.D4.C#5',
  'G4.C#5',
  'B3.D5',
  'B3.G4.D5',
  'B3.D4',
  'G4',
  'B3',
  'B3.G4',
  'B3.G4.E5',
  'B3.D4.D5.E5',
  'B3.D4.D5',
  'G4.C#5',
  'B3.D5',
  'B3.G4',
  'B3.D4.F5',
  'G4',
  'C4.E-5',
  'C4.G4.E-5',
  'C4.E4.E-5',
  'G4.E-5',


In [116]:
# Invert chord and duration dictionaries
intToChord = {i: c for c, i in chordToInt.items()}
intToDuration = {i: c for c, i in durationToInt.items()}

In [117]:
# Define sequence length
sequenceLength = 64

# Define empty arrays for train data
trainChords = []
trainDurations = []

In [118]:
# Construct training sequences for chords and durations
for s in range(len(cMajorChords)):
    chordList = [chordToInt[c] for c in cMajorChords[s]]
    durationList = [durationToInt[d] for d in cMajorDurations[s]]
    for i in range(len(chordList) - sequenceLength):
        trainChords.append(chordList[i:i+sequenceLength])
        trainDurations.append(durationList[i:i+sequenceLength])

Autoencoder

In [119]:
import tensorflow as tf

In [120]:
# Convert to one-hot encoding and swap chord and sequence dimensions
trainChords = tf.keras.utils.to_categorical(trainChords).transpose(0,2,1)

# Convert data to numpy array of type float
trainChords = np.array(trainChords, np.float)

In [121]:
# Define number of samples, chords and notes, and input dimension
nSamples = trainChords.shape[0]
nChords = trainChords.shape[1]
inputDim = nChords * sequenceLength

# Set number of latent features
latentDim = 2

In [122]:
# Flatten sequence of chords into single dimension
trainChordsFlat = trainChords.reshape(nSamples, inputDim)

In [123]:
# Define encoder input shape
encoderInput = tf.keras.layers.Input(shape = (inputDim))

# Define decoder input shape
latent = tf.keras.layers.Input(shape = (latentDim))

# Define dense encoding layer connecting input to latent vector
encoded = tf.keras.layers.Dense(latentDim, activation = 'tanh')(encoderInput)

# Define dense decoding layer connecting latent vector to output
decoded = tf.keras.layers.Dense(inputDim, activation = 'sigmoid')(latent)

# Define the encoder and decoder models
encoder = tf.keras.Model(encoderInput, encoded)
decoder = tf.keras.Model(latent, decoded)

# Define autoencoder model
autoencoder = tf.keras.Model(encoderInput, decoder(encoded))

In [124]:
# Compile autoencoder model
autoencoder.compile(loss = 'binary_crossentropy', optimizer='rmsprop')

# Train autoencoder
autoencoder.fit(trainChordsFlat, trainChordsFlat, epochs = 20)

Epoch 1/20
122/122 [==============================] - 36s 298ms/step - loss: 0.6162
Epoch 2/20
122/122 [==============================] - 33s 269ms/step - loss: 0.4636
Epoch 3/20
122/122 [==============================] - 31s 258ms/step - loss: 0.3482
Epoch 4/20
122/122 [==============================] - 30s 246ms/step - loss: 0.2604
Epoch 5/20
122/122 [==============================] - 32s 263ms/step - loss: 0.1941
Epoch 6/20
122/122 [==============================] - 30s 246ms/step - loss: 0.1448
Epoch 7/20
122/122 [==============================] - 30s 244ms/step - loss: 0.1085
Epoch 8/20
122/122 [==============================] - 30s 248ms/step - loss: 0.0820
Epoch 9/20
122/122 [==============================] - 31s 254ms/step - loss: 0.0627
Epoch 10/20
122/122 [==============================] - 31s 252ms/step - loss: 0.0487
Epoch 11/20
122/122 [==============================] - 32s 258ms/step - loss: 0.0385
Epoch 12/20
122/122 [==============================] - 35s 285ms/step - lo

Generation

In [128]:
# Generate chords from randomly generated latent vector
generatedChords = decoder(np.random.normal(size=(1,latentDim))).numpy().reshape(nChords, sequenceLength).argmax(0)


In [129]:
# Identify chord sequence from integer sequence
chordSequence = [intToChord[c] for c in generatedChords]

In [130]:
chordSequence 

['D5',
 'G4',
 'D5',
 'D5',
 'D5',
 'G4',
 'D5',
 'G4',
 'D5',
 'G4',
 'G4',
 'D5',
 'G4',
 'D5',
 'G4',
 'D5',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4',
 'G4']

In [66]:
# Set location to save generated music
generated_dir = 'generated'

from music21 import stream
# Generate stream with guitar as instrument
generatedStream = stream.Stream()
generatedStream.append(instrument.Piano())

In [67]:
# Append notes and chords to stream object
for j in range(len(chordSequence)):
    try:
        generatedStream.append(note.Note(chordSequence[j].replace('.', ' ')))
    except:
        generatedStream.append(chord.Chord(chordSequence[j].replace('.', ' ')))

In [68]:
generatedStream.write('midi', fp=generated_dir+'mozart.mid')

'generatedmozart.mid'